In [ ]:
### BIG ANALYSIS OF ALL FEEDSTOCKS SEPARATELY, MAY TAKE A WHILE TO RUN FULL-SCALE VERSION #####

import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# Mapping features to full English
varname_map = {
    'Population': 'Population',
    'Waste_per_capita': 'Waste per Capita',
    'WasteTotal': 'Total Waste',
    'recovery_H2_separation': 'H₂ Recovery (Separation)',
    'recovery_H2_storage': 'H₂ Recovery (Storage)',
    'eff_CGE': 'Cold Gas Efficiency',
    'eff_CCE': 'Carbon Conversion Efficiency',
    'ratio_H2_stored': 'H₂ Allocation to Storage',
    'ratio_H2_CHP': 'H₂ Allocation to CHP',
    'is_plasma_cleanup': 'Plasma Gas Cleaning',
    'fs_shape_pellets': 'Pellet Shape',
    'fs_shape_fibres': 'Fibre Shape',
    'fs_shape_dust': 'Dust Shape',
    'fs_shape_chips': 'Chip Shape',
    'fs_shape_particles': 'Particle Shape',
    'fs_shape_other': 'Other Shape',
    'fs_size': 'Feedstock Size',
    'fs_lhv': 'Feedstock LHV',
    'fs_C': 'Carbon Content',
    'fs_H': 'Hydrogen Content',
    'fs_N': 'Nitrogen Content',
    'fs_S': 'Sulphur Content',
    'fs_O': 'Oxygen Content',
    'fs_ash': 'Ash Content',
    'fs_moisture': 'Moisture Content',
    're_temp': 'Reactor Temperature',
    're_mode_continuous': 'Continuous Reactor',
    're_mode_batch': 'Batch Reactor',
    're_ER': 'Equivalence Ratio',
    're_steambiomass_ratio': 'Steam-to-Biomass Ratio',
    're_agent_air': 'Gasifying Agent: Air',
    're_agent_air_and_steam': 'Gasifying Agent: Air + Steam',
    're_agent_oxygen': 'Gasifying Agent: Oxygen',
    're_agent_steam': 'Gasifying Agent: Steam',
    're_agent_other': 'Gasifying Agent: Other',
    're_type_fluidised_bed': 'Fluidised Bed Reactor',
    're_type_fixed_bed': 'Fixed Bed Reactor',
    're_type_other': 'Other Reactor Type',
    're_material_olivine': 'Reactor Material: Olivine',
    're_material_silica': 'Reactor Material: Silica',
    're_material_dolomite': 'Reactor Material: Dolomite',
    're_material_alumina': 'Reactor Material: Alumina',
    're_material_calcium_oxide': 'Reactor Material: CaO',
    're_catalyst': 'Catalyst Present',
    're_scale_pilot': 'Pilot Scale',
    're_scale_lab': 'Lab Scale',
    'syn_N2': 'Syngas N₂',
    'syn_H2': 'Syngas H₂',
    'syn_CO': 'Syngas CO',
    'syn_CO2': 'Syngas CO₂',
    'syn_CH4': 'Syngas CH₄',
    'syn_C2Hn': 'Syngas C₂Hn',
    'syn_LHV': 'Syngas LHV',
    'syn_tar_content': 'Tar Content',
    'syn_yield': 'Syngas Yield',
    'syn_char_yield': 'Char Yield',
    'Feedstock_ID': 'Feedstock ID',
    'Iteration': 'Iteration',
    'Year': 'Year',
    'feedstock_type_HB': 'Herbaceous Biomass',
    'feedstock_type_MSW': 'Municipal Solid Waste',
    'feedstock_type_Other': 'Other Feedstock',
    'feedstock_type_Plastics': 'Plastics',
    'feedstock_type_WB': 'Woody Biomass'
}

# Load and preprocess input data
df = pd.read_csv("monte_carlo_results.csv")

columns_to_drop = [
    'syn_H2', 'syn_CO', 'syn_CO2', 'syn_CH4', 'syn_C2Hn', 'syn_N2',
    'syn_LHV', 'syn_tar_content', 'syn_yield', 'syn_char_yield'
]
df.drop(columns=columns_to_drop, errors='ignore', inplace=True)

feedstock_cols = [col for col in df.columns if col.startswith("feedstock_type_")]
feedstock_labels = [col.replace("feedstock_type_", "") for col in feedstock_cols]

df.drop(columns=["Feedstock_ID", "Year"], inplace=True)
groups = df["Iteration"]
X_df = df.drop(columns=["ANNUALENERGY_H2_kwh", "Iteration"])
y = df["ANNUALENERGY_H2_kwh"].values

for col in X_df.columns:
    if X_df[col].isnull().any():
        X_df[col] = X_df[col].fillna(X_df[col].median())

scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X_df)
feature_names = X_df.columns.tolist()
feature_labels = [varname_map.get(f, f) for f in feature_names]

# Load neural network model
model = load_model("best_model_allfeedstocks_NN1_v9.keras")

# Sample 500 representative rows across entire dataset
sample_count = 500
sample_indices = np.linspace(0, X_scaled.shape[0] - 1, sample_count, dtype=int)
X_sampled = X_scaled[sample_indices]

# SHAP on full dataset
explainer = shap.KernelExplainer(model.predict, X_sampled)
shap_values_list = explainer.shap_values(X_sampled)
shap_values_array = shap_values_list[0] if isinstance(shap_values_list, list) else shap_values_list

# Compute global importance ranking
importances = np.abs(shap_values_array).mean(axis=0)
importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Feature_Label": feature_labels,
    "Mean_Abs_SHAP": importances
}).sort_values(by="Mean_Abs_SHAP", ascending=False)

# Saving results
pd.DataFrame(shap_values_array, columns=feature_labels).to_csv("shap_values.csv", index=False)
pd.DataFrame(X_sampled, columns=feature_labels).to_csv("shap_inputs.csv", index=False)
importance_df.to_csv("shap_importance_ranking.csv", index=False)

# Plotting
def plot_summary(shap_vals, X_data, fname, max_display, plot_type='dot'):
    shap.summary_plot(
        shap_vals, features=X_data, feature_names=feature_labels,
        plot_type=plot_type, max_display=max_display, show=False
    )
    plt.tight_layout()
    plt.savefig(fname, dpi=300)
    plt.close()

# Global plots
plot_summary(shap_values_array, X_sampled, "SHAP_dot_top10.png", max_display=10, plot_type='dot')
plot_summary(shap_values_array, X_sampled, "SHAP_bar_top10.png", max_display=10, plot_type='bar')
plot_summary(shap_values_array, X_sampled, "SHAP_dot_top20.png", max_display=20, plot_type='dot')
plot_summary(shap_values_array, X_sampled, "SHAP_bar_top20.png", max_display=20, plot_type='bar')
plot_summary(shap_values_array, X_sampled, "SHAP_dot_all.png", max_display=len(feature_labels), plot_type='dot')
plot_summary(shap_values_array, X_sampled, "SHAP_bar_all.png", max_display=len(feature_labels), plot_type='bar')

# Per-feedstock plots
for col, label in zip(feedstock_cols, feedstock_labels):
    feedstock_df = df[df[col] == 1].copy()
    X_f = feedstock_df.drop(columns=["ANNUALENERGY_H2_kwh", "Iteration"])
    y_f = feedstock_df["ANNUALENERGY_H2_kwh"]

    for feat in X_f.columns:
        if X_f[feat].isnull().any():
            X_f[feat] = X_f[feat].fillna(X_f[feat].median())

    X_scaled_f = scaler_X.transform(X_f)
    sample_count_f = min(500, X_scaled_f.shape[0])  # cap to 500 or fewer if small dataset
    sample_idx = np.linspace(0, X_scaled_f.shape[0] - 1, sample_count_f, dtype=int)
    X_sampled_f = X_scaled_f[sample_idx]


    explainer_f = shap.KernelExplainer(model.predict, X_sampled_f)
    shap_vals_f = explainer_f.shap_values(X_sampled_f)
    shap_arr_f = shap_vals_f[0] if isinstance(shap_vals_f, list) else shap_vals_f

    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_dot_top5_{label}.png", max_display=5, plot_type='dot')
    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_bar_top5_{label}.png", max_display=5, plot_type='bar')
    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_dot_top10_{label}.png", max_display=10, plot_type='dot')
    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_bar_top10_{label}.png", max_display=10, plot_type='bar')
    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_dot_all_{label}.png", max_display=len(feature_labels), plot_type='dot')
    plot_summary(shap_arr_f, X_sampled_f, f"SHAP_bar_all_{label}.png", max_display=len(feature_labels), plot_type='bar')

# Confirmation statements
print(" SHAP 5-sample test complete.")
print("• Global plots: 6 files")
print("• Per-feedstock plots: 6 × 5 feedstocks = 30 files")
print("• CSVs saved with readable labels.")
